In [29]:
import pennylane as qml
from pennylane import numpy as np
from math import pi, sqrt, ceil
import sys

In [30]:
# Define the knapsack problem parameters
weights = np.array([2, 3, 2])
values = np.array([3, 1, 2])
max_weight = 4

# Number of items
n = len(weights)

# Define the device
dev = qml.device("default.qubit", wires=2 * n + 2)

# Define the Oracle function for Grover's algorithm
def oracle():
    # Mark the valid solutions
    for i in range(n):
        qml.Toffoli(wires=[i, n + i, 2 * n])
    for i in range(n):
        qml.CNOT(wires=[n + i, 2 * n + 1])

# Define the diffusion operator (Grover's diffusion operator)
def diffusion():
    for i in range(n):
        qml.Hadamard(wires=i)
        qml.PauliX(wires=i)
    qml.MultiControlledX(control_wires=list(range(n)), wires=2 * n, work_wires=[2 * n + 1])
    for i in range(n):
        qml.PauliX(wires=i)
        qml.Hadamard(wires=i)

# Define the quantum circuit
@qml.qnode(dev)
def grover_circuit():
    # Initialize superposition
    for i in range(n):
        qml.Hadamard(wires=i)

    # Apply Oracle and Diffusion Operator sqrt(N) times
    iterations = int(pi / 4 * sqrt(2 ** n))
    for _ in range(iterations):
        oracle()
        diffusion()

    return qml.probs(wires=range(n))

# Classical function to verify the validity of a solution
def is_valid(solution):
    weight = np.dot(weights, solution)
    value = np.dot(values, solution)
    if weight <= max_weight and value >= V:
        return True
    else:
        return False

In [31]:
print(qml.draw(grover_circuit)())

0: ──H─╭●──H──X──────────╭●──X──H─╭●──H──X──────────╭●──X──H─┤ ╭Probs
1: ──H─│──╭●──H──X───────├●──X──H─│──╭●──H──X───────├●──X──H─┤ ├Probs
2: ──H─│──│──╭●──H──X────├●──X──H─│──│──╭●──H──X────├●──X──H─┤ ╰Probs
3: ────├●─│──│──╭●───────│────────├●─│──│──╭●───────│────────┤       
4: ────│──├●─│──│──╭●────│────────│──├●─│──│──╭●────│────────┤       
5: ────│──│──├●─│──│──╭●─│────────│──│──├●─│──│──╭●─│────────┤       
6: ────╰X─╰X─╰X─│──│──│──╰X───────╰X─╰X─╰X─│──│──│──╰X───────┤       
7: ─────────────╰X─╰X─╰X───────────────────╰X─╰X─╰X──────────┤       


/home/ronin/miniforge3/envs/quantum/lib/python3.12/site-packages/pennylane/ops/op_math/controlled_ops.py:1046: UserWarning: The control_wires keyword will be removed soon. Use wires = (control_wires, target_wire) instead. See the documentation for more information.
  warnings.warn(


In [32]:
# Initialize parameters
Vmin = 0
Vmax = np.sum(values)
optimal_value = 0
optimal_solution = None

# Outer loop
while Vmin <= Vmax:
    V = np.floor((Vmin + Vmax) / 2)
    m = 2 ** n

    print(f"Vmin: {Vmin}, Vmax: {Vmax}, V: {V}")
    sys.exit()

    # Inner loop
    while m >= 1:
        probs = grover_circuit()
        most_probable_state = np.argmax(probs)
        solution = [int(bit) for bit in format(most_probable_state, f'0{n}b')]
        
        if is_valid(solution):
            optimal_solution = solution
            optimal_value = V
            Vmin = V + 1
            break
        else:
            m = m // 2

    if m < 1:
        Vmax = V - 1

# Print the optimal solution
print(f"Optimal solution: {optimal_solution}")
print(f"Optimal value: {optimal_value}")


Vmin: 0, Vmax: 6, V: 3.0


SystemExit: 

/home/ronin/miniforge3/envs/quantum/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3556: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
